In [13]:
from astropy.io import fits
import numpy as np
from scipy.interpolate import interp1d

# Open the FITS file
fits_file_path = '/home/ka/Downloads/fv5.5.2/tbmodel_20210827T210316000_20210827T210332000.fits'
with fits.open(fits_file_path) as hdul:
    # Inspect the structure of the FITS file
    hdul.info()

    # Access the SPECTRA and ENERGIES tables
    spectra_data = hdul[3].data
    energies_data = hdul[2].data

    # Print column names to identify correct ones
    print("SPECTRA Columns:", spectra_data.columns.names)
    print("ENERGIES Columns:", energies_data.columns.names)

    # Extract observed INTPSPEC data from SPECTRA and energies from ENERGIES table
    observed_intspec = spectra_data['INTPSPEC']  # INTPSPEC column for observed spectrum
    observed_energy_bins = (energies_data['ENERG_LO'] + energies_data['ENERG_HI']) / 2  # Average of low and high energy bounds

# Load your simulated data (scattered_photons or similar)
# Assuming 'scattered_photons' and 'simulated_energy_bins' are available in your script
simulated_energy_bins = np.array([1.0, 2.0, 3.0])  # Example energy bins for simulated data
scattered_photons = np.array([0.5, 0.7, 0.9])  # Example simulated data for scattered photons

# Interpolate scattered_photons to match the observed energy bins
interpolator = interp1d(simulated_energy_bins, scattered_photons, kind='linear', fill_value='extrapolate')
interpolated_scattered_photons = interpolator(observed_energy_bins)

# Compute ADNORM scaling factor
sum_observed = np.sum(observed_intspec)
sum_simulated = np.sum(interpolated_scattered_photons)
adnorm = sum_observed / sum_simulated

# Scale the simulated spectrum to match the observed data
scaled_intpspec = interpolated_scattered_photons * adnorm

# Print the results
print("ADNORM scaling factor:", adnorm)
print("Scaled INTPSPEC:", scaled_intpspec)


Filename: /home/ka/Downloads/fv5.5.2/tbmodel_20210827T210316000_20210827T210332000.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      11   ()      
  1  PARAMETERS    1 BinTableHDU     35   1R x 10C   [7A, 1I, 1I, 1E, 1I, 1I, 1E, 1E, 1I, 1I]   
  2  ENERGIES      1 BinTableHDU     17   3000R x 2C   [1E, 1E]   
  3  SPECTRA       1 BinTableHDU     17   15R x 2C   [1E, 3000E]   
SPECTRA Columns: ['paramval', 'INTPSPEC']
ENERGIES Columns: ['ENERG_LO', 'ENERG_HI']
ADNORM scaling factor: 3.1640448511589367
Scaled INTPSPEC: [ 1.0156584   1.02198648  1.02831458 ... 19.98094372 19.98727075
 19.99360019]


In [14]:
# Open the FITS file and list all extensions
hdul = fits.open(fits_file)
for hdu in hdul:
    print(hdu.name)
hdul.close()


PRIMARY
PARAMETERS
ENERGIES
SPECTRA


In [15]:
from astropy.io import fits
import numpy as np

# Load the observed data from the FITS file
fits_file = '/home/ka/Downloads/fv5.5.2/tbmodel_20210827T210316000_20210827T210332000.fits'
hdul = fits.open(fits_file)

# Retrieve SPECTRA and ENERGIES data
spectra_hdu = hdul['SPECTRA']
energies_hdu = hdul['ENERGIES']

# Extract the parameter and intensity data
paramval = spectra_hdu.data['paramval']
intpspec = spectra_hdu.data['INTPSPEC']
energy_lo = energies_hdu.data['ENERG_LO']
energy_hi = energies_hdu.data['ENERG_HI']

# Apply the ADNORM scaling factor
scaling_factor = 0.9640448511589367
scaled_intpspec = intpspec * scaling_factor

# Create the PRIMARY HDU (Header Data Unit) with basic metadata
primary_hdu = fits.PrimaryHDU()
primary_hdu.header['MODLNAME'] = 'scattered_solar'
primary_hdu.header['MODLUNIT'] = 'photons/cm^2/s'
primary_hdu.header['ADDMODEL'] = True
primary_hdu.header['HDUCLASS'] = 'OGIP'
primary_hdu.header['HDUCLAS1'] = 'XSPEC TABLE MODEL'
primary_hdu.header['HDUVERS'] = '1.1.0'

# Define PARAMETERS extension
param_data = np.array([
    ('scaling_factor', 0, 1.0, 0.1, 0.0, 0.0, 10.0, 10.0, 1, [1.0])
], dtype=[
    ('NAME', 'S12'), ('METHOD', 'i4'), ('INITIAL', 'f4'), ('DELTA', 'f4'),
    ('MINIMUM', 'f4'), ('BOTTOM', 'f4'), ('TOP', 'f4'), ('MAXIMUM', 'f4'),
    ('NUMBVALS', 'i4'), ('VALUE', 'f4', (1,))
])
param_cols = fits.ColDefs([
    fits.Column(name='NAME', format='12A', array=param_data['NAME']),
    fits.Column(name='METHOD', format='J', array=param_data['METHOD']),
    fits.Column(name='INITIAL', format='E', array=param_data['INITIAL']),
    fits.Column(name='DELTA', format='E', array=param_data['DELTA']),
    fits.Column(name='MINIMUM', format='E', array=param_data['MINIMUM']),
    fits.Column(name='BOTTOM', format='E', array=param_data['BOTTOM']),
    fits.Column(name='TOP', format='E', array=param_data['TOP']),
    fits.Column(name='MAXIMUM', format='E', array=param_data['MAXIMUM']),
    fits.Column(name='NUMBVALS', format='J', array=param_data['NUMBVALS']),
    fits.Column(name='VALUE', format='E', array=param_data['VALUE'])
])
param_hdu = fits.BinTableHDU.from_columns(param_cols)
param_hdu.header['HDUCLASS'] = 'OGIP'
param_hdu.header['HDUCLAS1'] = 'XSPEC TABLE MODEL'
param_hdu.header['HDUCLAS2'] = 'PARAMETERS'
param_hdu.header['HDUVERS'] = '1.0.0'

# Define ENERGIES extension (Energy bin limits)
col_energy = fits.ColDefs([
    fits.Column(name='ENERG_LO', format='E', array=energy_lo),
    fits.Column(name='ENERG_HI', format='E', array=energy_hi)
])
energy_hdu = fits.BinTableHDU.from_columns(col_energy)
energy_hdu.header['HDUCLASS'] = 'OGIP'
energy_hdu.header['HDUCLAS1'] = 'XSPEC TABLE MODEL'
energy_hdu.header['HDUCLAS2'] = 'ENERGIES'
energy_hdu.header['HDUVERS'] = '1.0.0'

# Define SPECTRA extension with PARAMVAL and scaled INTPSPEC
spectrum_cols = fits.ColDefs([
    fits.Column(name='PARAMVAL', format='E', array=paramval),  # PARAMVAL values from 0.0 to 1.4
    fits.Column(name='INTPSPEC', format=f'{intpspec.shape[1]}E', array=scaled_intpspec)  # 3000 energy bins
])
spectrum_hdu = fits.BinTableHDU.from_columns(spectrum_cols)
spectrum_hdu.header['HDUCLASS'] = 'OGIP'
spectrum_hdu.header['HDUCLAS1'] = 'XSPEC TABLE MODEL'
spectrum_hdu.header['HDUCLAS2'] = 'MODEL SPECTRA'
spectrum_hdu.header['HDUVERS'] = '1.0.0'

# Create HDU list and save the FITS file
output_path = '/home/ka/Downloads/fv5.5.2/scattered_solar_spectrum_model_v3.fits'
hdul = fits.HDUList([primary_hdu, param_hdu, energy_hdu, spectrum_hdu])
hdul.writeto(output_path, overwrite=True)

print(f"FITS file created successfully at '{output_path}'.")


FITS file created successfully at '/home/ka/Downloads/fv5.5.2/scattered_solar_spectrum_model_v3.fits'.
